# Tutorial 02: Control Flow

This tutorial introduces basic concepts in task-based parallel programming using Parla.
We will cover:
- TaskSpaces (NamedTasks)
- Task Dependencies
- Barrier synchronization

First, let's configure the tutorial environment.


In [65]:
from typing import Callable, Optional
from time import perf_counter, sleep

# Handle for Parla runtime
from parla import Parla

# Spawning  tasks
from parla.tasks import spawn, TaskSpace
from parla.devices import cpu


def run(function: Callable[[], Optional[TaskSpace]]):
    assert callable(function), "The function argument must be callable."

    # Start the Parla runtime.
    with Parla():
        # Create a top-level task to kick off the computation
        @spawn(placement=cpu, vcus=0)
        async def top_level_task():
            # Note that unless function returns a TaskSpace, this will NOT be blocking.
            # If you want to wait on the completion of the tasks launched by function, you must return a TaskSpace that contains their terminal tasks.
            await function()

    # Runtime exists at the end of the context_manager
    # All tasks are guaranteed to be complete at this point
    

## Referring to Tasks

### Unnamed Tasks. 
#### Example: `tutorial/04_unnamed_tasks.py`

As we saw in the first tutorial, we can create tasks using the `spawn` decorator.
If the task is not given a name explicitly, it is assigned a unique name in Parla's default global namespace. 

The function handle that was decorated by `@spawn` provides a reference to the task. 

When running code inside of a task, a reference to the active task can always be obtained using `parla.task.get_current_task()`.



In [5]:
from parla.tasks import get_current_task 

async def anonymous_tasks():
    list_of_tasks = []
    for i in range(4):
        @spawn()
        def task():
            my_name = get_current_task()
            print(f"Hello from Task {my_name}! \n", flush=True)
        
        list_of_tasks.append(task)
        
    sleep(0.1)
        
    print("List of Tasks: ", list_of_tasks, flush=True)

print("Running anonymous_tasks...")
run(anonymous_tasks)

Running anonymous_tasks...
Hello from Task Task(global_1)! 

Hello from Task Task(global_2)! 

Hello from Task Task(global_4)! 

Hello from Task Task(global_3)! 

List of Tasks:  [Task(global_1), Task(global_2), Task(global_3), Task(global_4)]



### TaskSpaces

**TaskSpaces** are an indexable collection of tasks. They provide a namespace for Task IDs. 

Every spawned task must have a **unique** identifier within the TaskSpace it is spawned in.
Two spawned tasks cannot be linked to the same underlying reference.  

#### Example: `tutorial/04_named_tasks.py`




In [6]:
from parla import TaskSpace

async def named_tasks():
    
    T = TaskSpace("T")
    n_tasks = 4
    for i in range(n_tasks):
        @spawn(T[i])
        def task():
            print(f"Hello from {T[i]}! \n", flush=True)
    sleep(0.1)
    print("TasksSpace: ", T)
    print("Contains: ", list(T.tasks), flush=True)
    
print("Running named_tasks...")
run(named_tasks)

Running named_tasks...
Hello from Task(T_0)! 

Hello from Task(T_2)! 

Hello from Task(T_3)! 

Hello from Task(T_1)! 

TasksSpace:  TaskSpace(T, ntasks=4)
Contains:  [Task(T_0), Task(T_1), Task(T_2), Task(T_3)]


TaskSpaces can be indexed by any hashable type. Although we recommend sticking to integers and slices for interpretability and performance.


#### Example: `tutorial/05_taskspace_slicing.py`
They can be sliced, are iterable, and can be indexed in arbitrary dimension.  

In [7]:
async def taskspace_slicing():
    T = TaskSpace("T")

    for i in range(2):
        for j in range(2):
            @spawn(T[i, j])
            def task():
                print(f"Hello from {T[i, j]}! \n", flush=True)
    
    sleep(0.1)
    print("TasksSpace: ", T)
    print("Slice of Tasks: ", T[0:1, 0:2], flush=True)
    print("State of Task[0, 0]: ", T[0, 0].state, flush=True)

print("Running taskspace_slicing...")
run(taskspace_slicing)

Running taskspace_slicing...
Hello from Task(T_0_0)! 

Hello from Task(T_1_1)! 

Hello from Task(T_1_0)! 

Hello from Task(T_0_1)! 

TasksSpace:  TaskSpace(T, ntasks=4)
Slice of Tasks:  TaskList: [Task(T_0_0), Task(T_0_1)]
State of Task[0, 0]:  TaskCompleted(None)



## Task Dependencies

Once a reference for a task is available it can be used to create dependencies between tasks.
This is the second argument to the `spawn` decorator.

Tasks can depend on any other task, in any TaskSpace.

Generally, tasks will not be launched until all of their dependencies have completed.

*Note: That this is not a strict requirement when allowing event driven submission of tasks to hardware queues (CUDA streams) "Runahead Scheduling".*


#### Example: `tutorial/06_dependencies.py`

Below we show a serial chain of tasks.

TaskSpaces have boundaries and a shape when indexed by integers. 
By default, a TaskSpace will only may only contain positive Task IDs,  [0, inf) along each dimension.
Indexes outside of the boundary will return empty references. 
This allows easy handling of base cases and boundary conditions.

In [8]:
async def serial_tasks():
    T = TaskSpace("T")

    for i in range(4):
        @spawn(T[i], dependencies=[T[i-1]]) # Could also have written dependencies=T[:i]
        def task():
            print(f"Hello from {T[i]}! \n", flush=True)
            
print("Running serial_tasks...")
run(serial_tasks)

Running serial_tasks...
Hello from Task(T_0)! 

Hello from Task(T_1)! 

Hello from Task(T_2)! 

Hello from Task(T_3)! 



#### Example: `tutorial/07_dependencies_reduction.py`

In [9]:
async def task_reduction_dependencies():
    import numpy as np

    T = TaskSpace("T")

    N = 8
    levels = int(np.log2(N))

    array = np.arange(N)
    expected_sum = np.sum(array)

    scratch = {}
    scratch[0] = array
    for level in range(1, levels):
        length = int(N / 2 ** (level + 1))
        scratch[level] = np.zeros(length)

    print("Initial array: ", array, flush=True)
    print("Expected sum: ", expected_sum, flush=True)

    # Generate tasks for a reduction tree
    for level in range(levels):
        stride = int(2 ** (level + 1))
        for idx in range(0, N, stride):
            writes_to = idx
            reads_from = idx + stride // 2

            @spawn(T[level, writes_to], [T[level - 1, reads_from]])
            def task():
                array[writes_to] += array[reads_from]

    # Wait for the reduction tree to finish
    await T[levels - 1, 0]

    print("Final array: ", array, flush=True)
    print("Sum: ", array[0], flush=True)


print("Running task_reduction_dependencies...")
run(task_reduction_dependencies)

Running task_reduction_dependencies...
Initial array:  [0 1 2 3 4 5 6 7]
Expected sum:  28
Final array:  [28  1  5  3 22  5 13  7]
Sum:  28


## Advanced TaskSpaces

As mentioned above, TaskSpaces can be given arbitrary shapes and boundaries.
This allows for more tunable handling and safety when indexing.

TaskSpace slicing is *dense*. It creates a handle for a task on first access.
This means that when indexing a TaskSpace, it will always return a reference to a task, even if it has not been created yet.
This is useful for creating dependencies on tasks that may not have been created yet.
Note that this can be dangerous if the task is never created, as it will create a hanging dependency that will never be satisfied.

#### Example: `tutorial/08_advanced_taskspace_boundaries.py`


In [21]:
async def taskspace_boundaries_and_access():
    T1 = TaskSpace("T1", shape=(2, 2))

    # TaskSpace with shape (2, 2) has 4 possible integer indices
    print("Shaped TaskSpace slicing on T1")
    print("TasksSpace: ", T1[:, :])
    print("Summary: ", T1)
    print("---")
    

    # TaskSpace slicing is *dense*. It creates a handle for a task on first access.
    T2 = TaskSpace("T2", shape=(2, 2))
    print("Handle Creation on T2")
    print("TasksSpace: ", T2[0, 0], T2[1, 1])
    print("Summary: ", T2)
    print("---")
    
    # Sparse access is possible with TaskSpace.view 
    # This will only return handles that already exist 
    print("Sparse Access on T2[:, :]")
    print(T2.view[:, :])


run(taskspace_boundaries_and_access)

Shaped TaskSpace slicing on T1
TasksSpace:  TaskList: [Task(T1_0_0), Task(T1_0_1), Task(T1_1_0), Task(T1_1_1)]
Summary:  TaskSpace(T1, ntasks=4)
---
Handle Creation on T2
TasksSpace:  Task(T2_0_0) Task(T2_1_1)
Summary:  TaskSpace(T2, ntasks=2)
---
Sparse Access on T2[:, :]
TaskList: [Task(T2_0_0), Task(T2_1_1)]


#### Example: `tutorial/09_out_of_order_spawn.py`

In [25]:
async def out_of_order():
    T = TaskSpace("T")
    
    @spawn(T[1], [T[0]])
    def task1():
        print("Hello from task1!", flush=True)
        
    @spawn(T[0])
    def task0():
        print("Hello from task0!", flush=True)


run(out_of_order)

Hello from task0!
Hello from task1!


# Barriers and Synchronization

Barriers are used to synchronize execution of tasks. They block a task's execution until all tasks listed in the barrier have completed.
Parla's barrier semantics are based on the Python async API model. 

This means any task that contains a barrier must be declared `async def` and spawned normally with `@spawn`.

Barriers are a special (implicit) type of dependency. 
When a task encounters a barrier, it will release control of its worker thread and spawn a new continuation of itself as a new separate task. 
This continuation task will depend on all tasks listed in the barrier and will not be launched until all of them have completed.

As a consequence, Barriers can only be used within tasks and cannot be used in the outermost non-task scope.
This is why our `run` function contains a top-level task. 

*Note: Reaching an await statement will release a tasks worker thread and non-persistent resources.*

In [66]:
async def simple_barrier():
    T = TaskSpace("T")
    
    for i in range(4):
        @spawn(T[i])
        def task1():
            print(f"Hello from task {i}!", flush=True)
            
        await T
        
run(simple_barrier)
        

Hello from task 0!
Hello from task 1!
Hello from task 2!
Hello from task 3!
